<a href="https://colab.research.google.com/github/eisbetterthanpi/vision/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title data
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
# https://github.com/python-engineer/pytorchTutorial/blob/master/14_cnn.py

# dataset has PILImage images of range [0, 1], transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# transform = transforms.Compose(transforms.ToTensor())

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
batch_size = 4
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_loader) # get some random training images
images, labels = dataiter.next()
# imshow(torchvision.utils.make_grid(images))


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# @title model
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv = nn.Sequential(
            # nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.Conv2d(3, 6, 5),
            # nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.linear = nn.Sequential(
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 16 * 5 * 5)
        x = self.linear(x)
        return x

class ConvNet1(nn.Module):
    def __init__(self):
        super(ConvNet1, self).__init__()
        plane_list=[64,128,256,512]
        self.conv = nn.Sequential( # nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0),
            nn.Conv2d(3, plane_list[0], 3, 1, 1), nn.BatchNorm2d(plane_list[0]), nn.ReLU(),# nn.MaxPool2d(2, 2),
            nn.Conv2d(plane_list[0], plane_list[1], 5, 1, 2), nn.BatchNorm2d(plane_list[1]), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(plane_list[1], plane_list[2], 7, 1, 3), nn.BatchNorm2d(plane_list[2]), nn.ReLU(), nn.MaxPool2d(2, 2),
        )
        self.linear = nn.Sequential(
            # nn.Linear(16 * 8 * 8, 256), nn.ReLU(),
            nn.Linear(plane_list[2]/16, 256), nn.ReLU(),
            nn.Linear(256, 64), nn.ReLU(),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.conv(x)
        print("forward x",x.shape)
        # x = x.view(-1, 16 * 5 * 5)
        x = nn.Flatten()(x)
        x = self.linear(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = ConvNet().to(device)
model = ConvNet1().to(device)
# print(model)

# X = torch.rand(64, 3, 32, 32, device=device)
# logits = model(X)
# print(logits.shape)
# print(logits[0])
# # print(logits[0].argmax(1))
# pred_probab = nn.Softmax(dim=1)(logits)
# y_pred = pred_probab.argmax(1)
# print(f"Predicted class: {y_pred}")


In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

def train():
    model.train()
    num_epochs = 5
    n_total_steps = len(train_loader)
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader): # origin shape: [4, 3, 32, 32] = 4, 3, 1024 input_layer: 3 input channels, 6 output channels, 5 kernel size
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if i % 2000 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
train()
print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

def test():
    model.eval()
    n_correct = 0
    n_samples = 0
    with torch.no_grad():
        n_class_correct = [0 for i in range(10)]
        n_class_samples = [0 for i in range(10)]
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            # max returns (value ,index)
            _, predicted = torch.max(outputs, 1)
            n_samples += labels.size(0)
            n_correct += (predicted == labels).sum().item()
            for i in range(batch_size):
                label = labels[i]
                pred = predicted[i]
                if (label == pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1
        acc = 100.0 * n_correct / n_samples
        print(f'Accuracy of the network: {acc} %')
        for i in range(10):
            acc = 100.0 * n_class_correct[i] / n_class_samples[i]
            print(f'Accuracy of {classes[i]}: {acc} %')

test()

Epoch [1/5], Step [1/12500], Loss: 0.2322
Epoch [1/5], Step [2001/12500], Loss: 0.1098
Epoch [1/5], Step [4001/12500], Loss: 0.3185
Epoch [1/5], Step [6001/12500], Loss: 0.0676
Epoch [1/5], Step [8001/12500], Loss: 0.2017
Epoch [1/5], Step [10001/12500], Loss: 0.7615
Epoch [1/5], Step [12001/12500], Loss: 0.0302
Epoch [2/5], Step [1/12500], Loss: 0.2153
Epoch [2/5], Step [2001/12500], Loss: 0.1599
Epoch [2/5], Step [4001/12500], Loss: 0.9032
Epoch [2/5], Step [6001/12500], Loss: 0.3563
Epoch [2/5], Step [8001/12500], Loss: 0.1930
Epoch [2/5], Step [10001/12500], Loss: 0.3855
Epoch [2/5], Step [12001/12500], Loss: 0.6684
Epoch [3/5], Step [1/12500], Loss: 0.0754
Epoch [3/5], Step [2001/12500], Loss: 0.3383
Epoch [3/5], Step [4001/12500], Loss: 1.0896
Epoch [3/5], Step [6001/12500], Loss: 0.5919
Epoch [3/5], Step [8001/12500], Loss: 0.0386
Epoch [3/5], Step [10001/12500], Loss: 0.7484
Epoch [3/5], Step [12001/12500], Loss: 0.4571
Epoch [4/5], Step [1/12500], Loss: 0.1541
Epoch [4/5], Ste

In [ ]:
# test()

'''
og
Epoch [5/5], Step [1/12500], Loss: 1.2949
Epoch [5/5], Step [2001/12500], Loss: 1.4118
Epoch [5/5], Step [4001/12500], Loss: 1.4136
Epoch [5/5], Step [6001/12500], Loss: 1.2962
Epoch [5/5], Step [8001/12500], Loss: 1.2799
Epoch [5/5], Step [10001/12500], Loss: 2.4948
Epoch [5/5], Step [12001/12500], Loss: 1.5663
Finished Training
Accuracy of the network: 50.5 %
Accuracy of plane: 50.1 %
Accuracy of car: 62.1 %
Accuracy of bird: 25.0 %
Accuracy of cat: 23.6 %
Accuracy of deer: 47.1 %
Accuracy of dog: 51.9 %
Accuracy of frog: 63.1 %
Accuracy of horse: 53.2 %
Accuracy of ship: 64.5 %
Accuracy of truck: 64.4 %

og rerun
Epoch [5/5], Step [1/12500], Loss: 1.4630
Epoch [5/5], Step [2001/12500], Loss: 1.2520
Epoch [5/5], Step [4001/12500], Loss: 1.5721
Epoch [5/5], Step [6001/12500], Loss: 0.9929
Epoch [5/5], Step [8001/12500], Loss: 1.5852
Epoch [5/5], Step [10001/12500], Loss: 0.8925
Epoch [5/5], Step [12001/12500], Loss: 1.3758
Finished Training
Accuracy of the network: 47.84 %
Accuracy of plane: 44.3 %
Accuracy of car: 72.4 %
Accuracy of bird: 34.9 %
Accuracy of cat: 16.0 %
Accuracy of deer: 31.7 %
Accuracy of dog: 58.9 %
Accuracy of frog: 59.0 %
Accuracy of horse: 60.4 %
Accuracy of ship: 51.9 %
Accuracy of truck: 48.9 %

new1
Epoch [5/5], Step [1/12500], Loss: 1.6666
Epoch [5/5], Step [2001/12500], Loss: 1.6778
Epoch [5/5], Step [4001/12500], Loss: 0.3968
Epoch [5/5], Step [6001/12500], Loss: 0.6304
Epoch [5/5], Step [8001/12500], Loss: 1.0713
Epoch [5/5], Step [10001/12500], Loss: 1.6662
Epoch [5/5], Step [12001/12500], Loss: 0.4982
Finished Training
Accuracy of the network: 65.52 %
Accuracy of plane: 66.5 %
Accuracy of car: 81.2 %
Accuracy of bird: 39.7 %
Accuracy of cat: 46.1 %
Accuracy of deer: 48.5 %
Accuracy of dog: 67.6 %
Accuracy of frog: 84.0 %
Accuracy of horse: 66.1 %
Accuracy of ship: 81.1 %
Accuracy of truck: 74.4 %

new1 rerun
Epoch [5/5], Step [1/12500], Loss: 1.6666
Epoch [5/5], Step [2001/12500], Loss: 1.6778
Epoch [5/5], Step [4001/12500], Loss: 0.3968
Epoch [5/5], Step [6001/12500], Loss: 0.6304
Epoch [5/5], Step [8001/12500], Loss: 1.0713
Epoch [5/5], Step [10001/12500], Loss: 1.6662
Epoch [5/5], Step [12001/12500], Loss: 0.4982
Finished Training
Accuracy of the network: 65.52 %
Accuracy of plane: 66.5 %
Accuracy of car: 81.2 %
Accuracy of bird: 39.7 %
Accuracy of cat: 46.1 %
Accuracy of deer: 48.5 %
Accuracy of dog: 67.6 %
Accuracy of frog: 84.0 %
Accuracy of horse: 66.1 %
Accuracy of ship: 81.1 %
Accuracy of truck: 74.4 %

new1 10
Epoch [5/5], Step [1/12500], Loss: 0.4570
Epoch [5/5], Step [2001/12500], Loss: 0.3988
Epoch [5/5], Step [4001/12500], Loss: 0.7006
Epoch [5/5], Step [6001/12500], Loss: 1.2796
Epoch [5/5], Step [8001/12500], Loss: 1.7792
Epoch [5/5], Step [10001/12500], Loss: 1.0257
Epoch [5/5], Step [12001/12500], Loss: 0.5086
Finished Training
Accuracy of the network: 70.14 %
Accuracy of plane: 73.7 %
Accuracy of car: 76.9 %
Accuracy of bird: 51.4 %
Accuracy of cat: 40.3 %
Accuracy of deer: 68.8 %
Accuracy of dog: 60.7 %
Accuracy of frog: 85.8 %
Accuracy of horse: 81.0 %
Accuracy of ship: 80.9 %
Accuracy of truck: 81.9 %

new1 15
Epoch [5/5], Step [1/12500], Loss: 0.0240
Epoch [5/5], Step [2001/12500], Loss: 1.0629
Epoch [5/5], Step [4001/12500], Loss: 0.5201
Epoch [5/5], Step [6001/12500], Loss: 0.2529
Epoch [5/5], Step [8001/12500], Loss: 1.1793
Epoch [5/5], Step [10001/12500], Loss: 0.1548
Epoch [5/5], Step [12001/12500], Loss: 0.7386
Finished Training
Accuracy of the network: 71.71 %
Accuracy of plane: 80.9 %
Accuracy of car: 85.1 %
Accuracy of bird: 57.8 %
Accuracy of cat: 44.2 %
Accuracy of deer: 74.7 %
Accuracy of dog: 62.5 %
Accuracy of frog: 84.3 %
Accuracy of horse: 76.2 %
Accuracy of ship: 74.8 %
Accuracy of truck: 76.6 %

new1 20
Epoch [5/5], Step [1/12500], Loss: 0.1975
Epoch [5/5], Step [2001/12500], Loss: 0.0396
Epoch [5/5], Step [4001/12500], Loss: 0.5252
Epoch [5/5], Step [6001/12500], Loss: 0.2810
Epoch [5/5], Step [8001/12500], Loss: 0.2805
Epoch [5/5], Step [10001/12500], Loss: 1.3831
Epoch [5/5], Step [12001/12500], Loss: 0.6854
Finished Training
Accuracy of the network: 71.3 %
Accuracy of plane: 69.7 %
Accuracy of car: 77.9 %
Accuracy of bird: 61.6 %
Accuracy of cat: 57.6 %
Accuracy of deer: 69.6 %
Accuracy of dog: 60.4 %
Accuracy of frog: 80.3 %
Accuracy of horse: 80.2 %
Accuracy of ship: 77.8 %
Accuracy of truck: 77.9 %

new1 25
Epoch [5/5], Step [1/12500], Loss: 0.0801
Epoch [5/5], Step [2001/12500], Loss: 0.1658
Epoch [5/5], Step [4001/12500], Loss: 0.0847
Epoch [5/5], Step [6001/12500], Loss: 0.1277
Epoch [5/5], Step [8001/12500], Loss: 0.4801
Epoch [5/5], Step [10001/12500], Loss: 0.1842
Epoch [5/5], Step [12001/12500], Loss: 0.0265
Finished Training
Accuracy of the network: 71.03 %
Accuracy of plane: 80.0 %
Accuracy of car: 84.4 %
Accuracy of bird: 61.2 %
Accuracy of cat: 44.9 %
Accuracy of deer: 69.8 %
Accuracy of dog: 59.5 %
Accuracy of frog: 73.9 %
Accuracy of horse: 83.1 %
Accuracy of ship: 78.3 %
Accuracy of truck: 75.2 %

'''
